In [2]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.models import Model

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import initializers
from keras.layers import regularizers
from keras.callbacks import EarlyStopping

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
 

from keras.optimizers import Adam, Adagrad, Adadelta

import keras.backend as bK
from keras.utils.np_utils import to_categorical

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import chi2, SelectKBest

# import statsmodels.formula.api as smf
# import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline


In [3]:
from numpy.random import seed
seed(123)
from tensorflow import set_random_seed
from tensorflow.contrib.layers import fully_connected 
import tensorflow as tf
set_random_seed(234)

In [4]:
data = pd.read_csv(r'C:\Users\jander33\Desktop\projects\project2\Production\raw data\WC_data\WC Data West only.csv')
df = data[data['New-Renew Ind'] != 'New']

df.reset_index(drop=True, inplace=True)
df.shape

(25706, 64)

In [62]:

test = df['Prime Agent Pd']
y = df['New-Renew Ind']
y = pd.get_dummies(y, drop_first = True)
labels = np.asarray(y)


In [64]:
token = Tokenizer() 

token.fit_on_texts(test)

sequences = token.texts_to_sequences(test) # list

In [65]:
# compare, just note that the cleaned data is now a keras object, Junk is not cleaned
print(sequences[0:2]) # this is a list
print(test[0:2])

[[55, 119], [714, 5, 3]]
0    WESTERN ASSURANCE
1    DALTON AGENCY INC
Name: Prime Agent Pd, dtype: object


In [66]:
word_index = token.word_index # type = dict
print('Found {} unique tokens.'.format(len(word_index)))
# max features = 1272 + 1

Found 1237 unique tokens.


In [67]:
data = pad_sequences(sequences, maxlen = None) 
#numpyarray of dim (samples, maxlen)
data[0:4]

array([[   0,    0,    0,    0,   55,  119],
       [   0,    0,    0,  714,    5,    3],
       [   0,    0, 1172, 1173,    2,    3],
       [   0,    0,  283,    1,    8,    4]])

In [68]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (25706, 6)
Shape of label tensor: (25706, 1)


In [81]:
rev_word_index = {v+3:k for k,v in word_index.items()}
rev_word_index[0] = 'padding_char'
rev_word_index[1] = 'start_char'
rev_word_index[2] = 'oov_char'
rev_word_index[3] = 'unk_char'

In [82]:
example_feat = ' '.join([rev_word_index[word] for word in data[2]])
example_feat #notice the zeros -> this is the padding. All lowerc

'padding_char padding_char webb fc oov_char unk_char'

In [87]:
X_train, X_test, y_train, y_test = train_test_split(data, labels,
                                                    test_size=0.3)

In [88]:
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)

(17994, 6)
(7712, 6)
(7712, 1)
(17994, 1)


In [103]:
X_train[0:3]

array([[  0,   0,  15, 122, 433,   2],
       [  0,   0,   0, 241, 206,   4],
       [  0,   0, 281,  21,   1,   3]])

In [97]:
max_features = max([max(x) for x in X_train] + 
                   [max(x) for x in X_test]) + 1
max_features # 45

1238

In [98]:
bK.clear_session()

In [102]:
# model = Sequential()
# model.add(Embedding(max_features, 64))
# model.compile('rmsprop', 'mse')

In [100]:
input_i = Input(shape=(6, ))

1238

shape = batches of 6 dimensional vectors

In [106]:
num_inputs = 6  # 6 dimensional input
num_hidden = 1  # 1 dimensional representation 
num_outputs = num_inputs # Must be true for an autoencoder!

learning_rate = 0.01

In [107]:
# Placeholder
X = tf.placeholder(tf.float32, shape=[None, num_inputs])

In [108]:
# Layers
hidden = fully_connected(X, num_hidden, activation_fn=None)
outputs = fully_connected(hidden, num_outputs, activation_fn=None)

In [109]:
# Loss Function
loss = tf.reduce_mean(tf.square(outputs - X))  # MSE

In [110]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train  = optimizer.minimize( loss)

In [113]:
init = tf.global_variables_initializer()

In [114]:
num_steps = 1000

with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(num_steps):
        sess.run(train,feed_dict={X: data})

In [115]:
with tf.Session() as sess:
    sess.run(init)
        
    # Now ask for the hidden layer output (the 2 dimensional output)
    output_1d = hidden.eval(feed_dict={X: data})

In [118]:
output_1d[0:18]

array([[  -64.22485352],
       [  547.69403076],
       [ 1281.24291992],
       [   93.71153259],
       [  554.74285889],
       [  185.24230957],
       [ -222.40309143],
       [  151.62176514],
       [  -11.93527603],
       [  449.03607178],
       [   42.47973251],
       [  146.42036438],
       [  165.66860962],
       [  665.9586792 ],
       [   -5.61925697],
       [   54.179245  ],
       [   79.79898071],
       [  176.08056641]], dtype=float32)

In [ ]:

model = Sequential()
model.add(Embedding(len(vocabulary), 300))
model.compile('rmsprop', 'mse')

input_i = Input(shape=(30, 300))
encoded_h1 = Dense(64, activation='tanh')(input_i)
encoded_h2 = Dense(32, activation='tanh')(encoded_h1)
encoded_h3 = Dense(16, activation='tanh')(encoded_h2)
encoded_h4 = Dense(8, activation='tanh')(encoded_h3)
encoded_h5 = Dense(4, activation='tanh')(encoded_h4)
latent = Dense(2, activation='tanh')(encoded_h5)
decoder_h1 = Dense(4, activation='tanh')(latent)
decoder_h2 = Dense(8, activation='tanh')(decoder_h1)
decoder_h3 = Dense(16, activation='tanh')(decoder_h2)
decoder_h4 = Dense(32, activation='tanh')(decoder_h3)
decoder_h5 = Dense(64, activation='tanh')(decoder_h4)

output = Dense(300, activation='tanh')(decoder_h5)

autoencoder = Model(input_i,output)

autoencoder.compile('adadelta','mse')

X_embedded = model.predict(X_train)
autoencoder.fit(X_embedded,X_embedded,epochs=10, batch_size=256, validation_split=.1)

print autoencoder.summary()

In [95]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(max_features, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [96]:
model.fit(X_train, y_train,
          batch_size=32,
          epochs=20,
          callbacks=[EarlyStopping(monitor='acc', patience=1)],
          validation_split=0.3)

ValueError: Error when checking target: expected dense_1 to have shape (1238,) but got array with shape (1,)

## Dict mapping

In [ ]:
def code_dict(col, dframe):
    
    '''returns a dict for nominals. This can then be mapped to 
    any col to create coded feature'''
    
    from collections import defaultdict
    
    #df = pd.DataFrame(dframe[col].value_counts())
    #df.reset_index(level=0, inplace=True)
    #df.rename(columns={'index': 'key', col: 'value'}, inplace=True)
#     df_dict = defaultdict(list)
#     for k, v in zip(df.key, df.value):
#         df_dict[k] = (int(v))
    return df_dict
        

In [1]:
UW2rank = {'Mulloy,Spencer': 23,       
'Despain,Royd': 22,         
'Rodlin,Debbie': 21,      
'Armstrong,Ryan': 20,       
'Moralez,Randa': 19,        
'Dunn,Jonathan': 18,        
'Carmicheal,Debra': 17,     
'Chaudhuri,Debbie': 16,     
'Fuss,Amanda': 15,        
'Reeves,Heather': 14,      
'Harding,Kristine': 13,      
'Harris,Norm': 12,          
'Ruggeberg,Mike': 11,      
'Moore,David': 10,         
'Hilst,Scott': 9,            
'Furness,Nick': 8,          
'White,Becky': 7,           
'Dole,Kylie': 6,           
'Denney,Melanie': 5,         
'Eastwood,Jennifer': 4,      
'Costello,Cathleen': 3,      
'Sebolt,Catie': 2,           
'Rybeck,Chris': 1,           
}

### Ordinal converters

In [ ]:
def make_dict(col, dframe):
    
    '''returns a dict for freqs. This can then be mapped to 
    any col to create freq feature'''
    
    from collections import defaultdict
    
    df = pd.DataFrame(dframe[col].value_counts())
    df.reset_index(level=0, inplace=True)
    df.rename(columns={'index': 'key', col: 'value'}, inplace=True)
    df_dict = defaultdict(list)
    for k, v in zip(df.key, df.value):
        df_dict[k] = (int(v))
    return df_dict
        

In [ ]:
def freq_group(freq, _dict, rare, infrequent, less_common):
    
    rev_dict = {v:k for k, v in _dict.items()}
    
    if freq <= rare:
        string = 'rare'
    elif freq > rare and freq <= infrequent:
        string = 'infrequent'
    elif freq > infrequent and freq <= less_common:
        string = 'less common'
    else:
        string = rev_dict[freq]
    return(string)

# policy lifetime feature

In [ ]:
pol_life =  pd.concat([df['Effective YearMonth'].apply(lambda x: str(x)), 
                   df['renew'], df['tenure'].apply(lambda x: str(x)),
                   ], axis=1)
pol_life = pol_life.apply(lambda x: ' '.join(x), axis=1) 

period = datetime(year=2018, month=7, day=1)

In [ ]:
​def delta(string, period):
    
    ''' The following is needed before running this:
    
        pol_life =  pd.concat([df['Effective YearMonth'].apply(lambda x: str(x)), 
                   df['renew'], df['tenure'].apply(lambda x: str(x)),
                   ], axis=1)
                   
        pol_life = pol_life.apply(lambda x: ' '.join(x), axis=1) 
        period = datetime(year=2018, month=7, day=1) 
        
        nan are good in this output. The function grabs the most recent renewal for a 
        certain line account and nans the other ones.'''
    
    eff = datetime.strptime(string.split()[0],'%Y%m')
    tenure = int(float(string.split()[2]))
    while string.split()[1] == 'Renew' and eff > period.replace(year=int(period.year - 1)):
        
        '''Take it back two years. Most of the violaters have more tenure.
        Renews with tenure = 1 are over 1 year in lifetime. The nans are a good thing. 
        They are the prior renewals on the same policy'''
            
        for i in range(1, 32):
            #print(i)
            # start with tenure 2

            if tenure == i:
                delta = period - eff
                days = delta.days + 365*i
                return(days)
            else:
                continue                

        break
        
    while string.split()[1] == 'New':
        
        if eff >= period.replace(year=int(period.year - 1)):
            delta = period - eff
            days = delta.days
            return(days)
    
        else: 
            days = 'remove'
            return(days)
        break
 

    while string.split()[1] == 'Non-Renew':
        
        days = tenure*365
        return(days)
        break



           

In [ ]:
life = pol_life.apply(lambda x: delta(x, period))